In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Importing data

In [ ]:
# Dataset folders 
dataset_dir = '/kaggle/input/an2dl-challenge-1-nt/data_splitted_no_test'
training_dir = os.path.join(dataset_dir, 'train')
validation_dir = os.path.join(dataset_dir, 'val')

In [ ]:
# Images are divided into folders, one for each class. 
# If the images are organized in such a way, we can exploit the 
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator, and for the trainign with Data Augmentation
train_data_gen = ImageDataGenerator(rotation_range=30, 
                                    height_shift_range=0.2, 
                                    width_shift_range=0.2, 
                                    zoom_range=0.2, 
                                    horizontal_flip=True, 
                                    shear_range=0.2, 
                                    fill_mode='reflect',
                                    rescale=1/255.) # rescale value is multiplied to the image ############################################
valid_data_gen = ImageDataGenerator(rescale=1/255.)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=8,
                                                       shuffle=True,
                                                       seed=seed)

valid_gen = valid_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)

## Model

In [ ]:
import os
os.listdir("/kaggle/input/ir-v102/IR_v1.0.2")

In [ ]:
# Re-load the model after transfer learning
IR = tfk.models.load_model('/kaggle/input/ir-v102/IR_v1.0.2')
IR.summary()

In [ ]:
# to create a folder
import os
directory = "InceptionResNetV2"
parent_dir = "/kaggle/working"
path = os.path.join(parent_dir, directory)
os.mkdir(path)

In [ ]:
!cd /kaggle/working/InceptionResNetV2

In [ ]:
# Set all VGG layers to True
IR.get_layer('inception_resnet_v2').trainable = True

In [ ]:
'''for i, layer in enumerate(IR.get_layer('inception_resnet_v2').layers):
   print(i, layer.name, layer.trainable)'''

In [ ]:
# Set VGG layers between 40 and 227 to False
'''for i, layer in enumerate(IR.get_layer('inception_resnet_v2').layers[:394]):
  layer.trainable=False
IR.summary()'''

In [ ]:
# Compile the model
IR.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.RMSprop(learning_rate=0.0001), metrics='accuracy')

In [ ]:
# Train the model
IR_history = IR.fit(
    x = train_gen,
    batch_size = 256,
    epochs = 200,
    validation_data = valid_gen,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=30, restore_best_weights=True)],
).history

In [ ]:
# Save the best model
IR.save('IR_v1.0.4')

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(IR_history['loss'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(IR_history['val_loss'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)
plt.savefig('IR_v1.0.4_loss.png')

plt.figure(figsize=(15,5))
plt.plot(IR_history['accuracy'], alpha=.3, color='#ff7f0e', linestyle='--')
plt.plot(IR_history['val_accuracy'], label='Standard', alpha=.8, color='#ff7f0e')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)
plt.savefig('IR_v1.0.4_acc.png')

plt.show()

In [ ]:
# Evaluate on val
val_metrics = IR.evaluate(valid_gen, return_dict=True)

print()
print("Val metrics")
print(val_metrics)